## Installing the library (ortools) and Uploading Data

In [1]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.2

## We need to import three files:
  1.  **```Student Registration Data.csv ```**- This file contains the student-course mapping data.<br>

  2. **```Course Offered List.csv```** - We use this file to get the course-professor mappings, by left joining with the ```G CODE``` of the ```Student Reegistration Data.csv```. <br>

  3. **```Faculty Preference.csv```** - This file contains the information about the busy slots / not preferred slots of the professors, which is used to filter out the timeslots before feeding it in the ```cp_model```



In [2]:
# Uploading the student-registration data
from google.colab import files
uploaded = files.upload()

Saving faculty_pref.csv to faculty_pref.csv
Saving Student Registration Data.csv to Student Registration Data.csv
Saving Students - Courses Offere List - AY 2024-25 - Term 1 - Sheet1.csv to Students - Courses Offere List - AY 2024-25 - Term 1 - Sheet1.csv


In [3]:
# Reading the database
import numpy as np
import pandas as pd

df_registration = pd.read_csv('Student Registration Data.csv') # Student Registration Data (Student-Course Mappings)
df_courses  = pd.read_csv('Students - Courses Offere List - AY 2024-25 - Term 1 - Sheet1.csv') # Course Offered List (Course-Professor Mappings)
df_faculty_pref = pd.read_csv('faculty_pref.csv') # Faculty Preference Data

Now we merge the two dataframes, df_registration and df_courses, based on the columns G CODE and Course code respectively.

In this case, it allows us to associate student registration data with course information, including the professor's name.

In [4]:
# Extracting the Course-Professor Mappin
df_merged = pd.merge(df_registration, df_courses, left_on='G CODE', right_on='Course code', how='left')
df_merged['Professor'] = df_merged['Faculty Name']

df = df_merged[['Roll No.', 'TRIM', 'SECTION', 'G CODE', 'Sections', 'Professor']]
df = df.dropna()
df.head()

,Roll No.,TRIM,SECTION,G CODE,Sections,Professor
0,UG-T4-001,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi
1,UG-T4-004,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi
2,UG-T4-006,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi
3,UG-T4-007,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi
4,UG-T4-010,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi


## Course by Student
Outputs the list of courses taken by the student, given the rollno.

In [5]:
def courses_by_students(rollno, df):
  """
  This function is designed to output the courses taken by the repective student.

  Inputs:
  rollno - roll number of the student (unique ID)
  df - database containing the registration data

  Outputs:
  student_courses - list of courses taken by the student
  """
  student_courses = df[df['Roll No.'] == rollno]['G CODE'].tolist()
  return student_courses

In [6]:
# This code snippet generates a dictionary to map each student to their enrolled courses.
# {rollno: [course1, course2, ... ,], ...}

roll_unique = df['Roll No.'].unique()
student_course_map  = {}
for roll in roll_unique:
  student_course_map[roll] = courses_by_students(roll, df)

## Handling the faculty preference

To make it readable, we will insert a new column ```Busy Slot``` in the ```Student Registration Dataset``` and account for busy slots associated with the respective professors.

In [7]:
# Diactionary with keys as the professor names and values as a list containing the busy slots.
faculty_busy = df_faculty_pref.groupby("Name")["Busy Slot"].agg(list).to_dict()

# Adding Busy Slot in the dataframe (as a list).
df["Busy Slot"] = df["Professor"].apply(lambda prof: faculty_busy.get(prof, []))

In [8]:
# Course Professor Mapping
course_professor_map = pd.Series(df['Professor'].values, index=df['G CODE']).to_dict()

In [9]:
# Testing is everything works right.
df

,Roll No.,TRIM,SECTION,G CODE,Sections,Professor,Busy Slot
0,UG-T4-001,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi,[]
1,UG-T4-004,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi,[]
2,UG-T4-006,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi,[]
3,UG-T4-007,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi,[]
4,UG-T4-010,4,BUSI203,BUSI203,A,Swayam Sampurna Panigrahi,[]
...,...,...,...,...,...,...,...
2977,PhD-T4-002,PhD4,ENVS305,ENVS305,A,Jessica Seddon,"[Monday 08:30, Monday 10:30, Monday 12:30, Mon..."
2978,PhD-T4-002,PhD4,ENVS310,ENVS310,A,Jaideep Hardikar,[]
2979,PhD-T4-001,PhD4,SOCL215,SOCL215|POLT222,A,Panchali Ray,[]
2981,UG-T7-145,7,BIOS251,BIOS251|HIST251,A,John Mathew,[]


## Time Slots Mapping

In [10]:
def create_course_dictionary(student_course_map, course_professor_map, professor_busy_slots):
    """
    Create a dictionary mapping each unique course to a dictionary containing potential time slots,
    excluding those when the course's professor is busy.

    Parameters:
    - student_course_map (dict): A dictionary mapping student identifiers to the courses they are enrolled in.
    - course_professor_map (dict): A dictionary mapping each course to its respective professor.
    - professor_busy_slots (dict): A dictionary mapping professors to their busy slots.

    Returns:
    - dict: A dictionary where each key is a course name, and the value is another dictionary with a key 'time_slots',
      listing all available time slots for that course, excluding when the professor is busy.
    """
    unique_courses = set()
    for courses in student_course_map.values():
        unique_courses.update(courses)

    course_availability = {}
    for course in unique_courses:
        professor = course_professor_map.get(course)
        busy_slots = professor_busy_slots.get(professor, [])

        # Generate all possible time slots and exclude busy ones
        all_slots = [
            f'{day} {time}' for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
            for time in ['08:30', '10:30', '12:30', '14:30', '16:30', '18:30']
        ]

        available_slots = [slot for slot in all_slots if slot not in busy_slots]
        course_availability[course] = {'time_slots': available_slots}

    return course_availability


## CP Model

In [11]:
import re
from ortools.sat.python import cp_model

def schedule_courses(courses, student_course_map):
    model = cp_model.CpModel()

    # Variables for each course and time slot
    course_time_vars = {}
    course_day_vars = {}  # Tracks days for courses
    days = ['Monday ', 'Tuesday ', 'Wednesday ', 'Thursday ', 'Friday ']

    # Create model variables
    for course_id, course_info in courses.items():
        course_time_vars[course_id] = []
        course_day_vars[course_id] = {day: [] for day in days}

        for time_slot in course_info['time_slots']:
            var_id = f'{course_id}_{time_slot}'
            var = model.NewBoolVar(var_id)
            course_time_vars[course_id].append(var)

            # Extract the day from time_slot using a regular expression
            match = re.match(r"(\D+)", time_slot)
            if match:
                day = match.group(1)
                if day in days:
                    course_day_vars[course_id][day].append(var)
                else:
                    print(f"Day extracted '{day}' is not recognized as a valid day.")

    # Constraint: Each course is scheduled exactly twice
    for course_id, vars in course_time_vars.items():
        model.Add(sum(vars) == 2)

    # No course is scheduled more than once per day
    for course_id, day_vars in course_day_vars.items():
        for day, vars in day_vars.items():
            model.Add(sum(vars) <= 1)

    # No student should have overlapping courses
    for roll_number, course_list in student_course_map.items():
        for i in range(len(course_list) - 1):
            for j in range(i + 1, len(course_list)):
                course1 = course_list[i]
                course2 = course_list[j]
                for k, time_slot1 in enumerate(courses[course1]['time_slots']):
                    for l, time_slot2 in enumerate(courses[course2]['time_slots']):
                        if time_slot1 == time_slot2:
                            model.AddBoolOr([
                                course_time_vars[course1][k].Not(),
                                course_time_vars[course2][l].Not()
                            ])

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status == cp_model.OPTIMAL:
        print("Solution found!")
        schedule_output = []
        for course_id, vars in course_time_vars.items():
            scheduled_times = [var.Name().split('_')[1] for var in vars if solver.Value(var)]
            schedule_output.append(f"{course_id} is scheduled at: {', '.join(scheduled_times)}")
        schedule_string = "\n".join(schedule_output)
        return schedule_string
    else:
        print("No solution exists.")
        return "No solution exists."




In [12]:
# Execution
courses = create_course_dictionary(student_course_map, course_professor_map, faculty_busy)
schedule_data = schedule_courses(courses, student_course_map)
if schedule_data:
    print("Scheduling completed successfully.")
else:
    print("Failed to generate a schedule.")


Solution found!
Scheduling completed successfully.


In [13]:
print(schedule_data)

PHYS222 is scheduled at: Monday 14:30, Tuesday 16:30
PHYS227 is scheduled at: Thursday 16:30, Friday 16:30
ARTS201 is scheduled at: Thursday 18:30, Friday 10:30
BIOS235|ENVS236 is scheduled at: Tuesday 12:30, Wednesday 16:30
PHIL202 is scheduled at: Monday 16:30, Friday 14:30
MATH304 is scheduled at: Thursday 18:30, Friday 08:30
BIOS251|HIST251 is scheduled at: Monday 18:30, Wednesday 14:30
SOCL215|POLT222 is scheduled at: Monday 16:30, Friday 18:30
CHEM215 is scheduled at: Wednesday 18:30, Thursday 12:30
ENVS206 is scheduled at: Monday 08:30, Tuesday 16:30
POLT203 is scheduled at: Monday 10:30, Wednesday 08:30
ECON313 is scheduled at: Wednesday 12:30, Thursday 16:30
HIST337 is scheduled at: Wednesday 10:30, Friday 08:30
PHYS211|COMP211 is scheduled at: Wednesday 14:30, Thursday 16:30
POLT317 is scheduled at: Monday 14:30, Tuesday 10:30
BIOS205 is scheduled at: Monday 14:30, Tuesday 10:30
ENVS305 is scheduled at: Thursday 14:30, Friday 14:30
PSYC226 is scheduled at: Monday 08:30, Wedne

## Testing

First we need to format it better.We create a dictionary with keys as the ```Course Code``` and values as the ```Time Slots``` it's scheduled on.

It will look like this:
```{Course_Name : [TimeSlot1, TimeSlot2]}```

In [14]:
# Formatting it into a Dictionary
schedule = {}
lines = schedule_data.strip().split('\n')
for line in lines:
    parts = line.split(' is scheduled at: ')
    course = parts[0].strip()
    times = parts[1].strip().split(', ')
    schedule[course] = times

In [19]:
schedule

{'PHYS222': ['Monday 14:30', 'Tuesday 16:30'],
 'PHYS227': ['Thursday 16:30', 'Friday 16:30'],
 'ARTS201': ['Thursday 18:30', 'Friday 10:30'],
 'BIOS235|ENVS236': ['Tuesday 12:30', 'Wednesday 16:30'],
 'PHIL202': ['Monday 16:30', 'Friday 14:30'],
 'MATH304': ['Thursday 18:30', 'Friday 08:30'],
 'BIOS251|HIST251': ['Monday 18:30', 'Wednesday 14:30'],
 'SOCL215|POLT222': ['Monday 16:30', 'Friday 18:30'],
 'CHEM215': ['Wednesday 18:30', 'Thursday 12:30'],
 'ENVS206': ['Monday 08:30', 'Tuesday 16:30'],
 'POLT203': ['Monday 10:30', 'Wednesday 08:30'],
 'ECON313': ['Wednesday 12:30', 'Thursday 16:30'],
 'HIST337': ['Wednesday 10:30', 'Friday 08:30'],
 'PHYS211|COMP211': ['Wednesday 14:30', 'Thursday 16:30'],
 'POLT317': ['Monday 14:30', 'Tuesday 10:30'],
 'BIOS205': ['Monday 14:30', 'Tuesday 10:30'],
 'ENVS305': ['Thursday 14:30', 'Friday 14:30'],
 'PSYC226': ['Monday 08:30', 'Wednesday 10:30'],
 'BIOS224': ['Monday 08:30', 'Friday 12:30'],
 'DATA231|COMP231': ['Thursday 14:30', 'Friday 14:3

Next we prepare for testing. We progress to create a dictionary with the ```Time Slots``` as the keys and ```Course Scheduled``` as the values in a list.

In [15]:
# Time Slots Comprehension
days = ['Monday ', 'Tuesday ', 'Wednesday ', 'Thursday ', 'Friday '] # Working Days
time = ['08:30', '10:30', '12:30', '14:30', '16:30', '18:30'] # Time Slots
all = [i+j for i in days for j in time] # Concatenated Time Slots

# Dictionnary of time slots as keys, with empty lists as values for now
time_slots = {}
for i in all:
  time_slots[i] = []

# Appending the appropraiate course names, as per the schedule.
for i in schedule:
  for j in schedule[i]:
    time_slots[j].append(i)


In this code, we are checking for scheduling clashes among students across different time slots. The procedure involves iterating through each time slot, gathering the list of students assigned to classes in that slot, and checking for any duplicates (indicating a clash).

In [16]:
temp_list = []

for i in time_slots:
  for j in time_slots[i]:
    temp_list += df[df['G CODE'] == j]['Roll No.'].tolist()
    if len(temp_list) != len(set(temp_list)):
      print(f'Clash found on {i}')
    else:
      print(f'No clash found on {i}')
    temp_list = []




No clash found on Monday 08:30
No clash found on Monday 08:30
No clash found on Monday 08:30
No clash found on Monday 08:30
No clash found on Monday 08:30
No clash found on Monday 08:30
No clash found on Monday 08:30
No clash found on Monday 08:30
No clash found on Monday 10:30
No clash found on Monday 10:30
No clash found on Monday 10:30
No clash found on Monday 10:30
No clash found on Monday 12:30
No clash found on Monday 12:30
No clash found on Monday 12:30
No clash found on Monday 12:30
No clash found on Monday 12:30
No clash found on Monday 14:30
No clash found on Monday 14:30
No clash found on Monday 14:30
No clash found on Monday 16:30
No clash found on Monday 16:30
No clash found on Monday 16:30
No clash found on Monday 16:30
No clash found on Monday 16:30
No clash found on Monday 18:30
No clash found on Monday 18:30
No clash found on Monday 18:30
No clash found on Monday 18:30
No clash found on Monday 18:30
No clash found on Monday 18:30
No clash found on Tuesday 08:30
No clas

In [17]:
import pandas as pd

# Define time slots based on typical class hours
time_slots1 = ['08:30', '10:30', '12:30', '14:30', '16:30', '18:30']

# Days of the week
days_of_week1 = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Create a DataFrame with these time slots
output = pd.DataFrame(index=time_slots1, columns=days_of_week1)


# Populate the DataFrame
for course, times in schedule.items():
    for time in times:
        day, slot = time.split()
        output.at[slot, day] = course if pd.isna(output.at[slot, day]) else output.at[slot, day] + "; " + course

# Output to Excel
output.to_excel("University_Timetable.xlsx")


## Check Conflicts

In [18]:
def check_conflicts(schedule, student_course_map):
    # Initialize a dictionary to hold each student's schedule for validation
    student_schedules = {student: [] for student in student_course_map}

    # Populate the student schedules with times from the course schedule
    for student, courses in student_course_map.items():
        for course in courses:
            course = course.split('|')[0]  # Adjust if course names contain extra identifiers
            if course in schedule:
                student_schedules[student].extend(schedule[course])

    # Check each student's schedule for any overlap
    conflicts = {}
    for student, times in student_schedules.items():
        if len(times) != len(set(times)):  # There's a duplicate time entry, hence a conflict
            conflicts[student] = times

    return conflicts

# Example usage
conflicts = check_conflicts(schedule, student_course_map)
if conflicts:
    print("Conflicts found:")
    for student, times in conflicts.items():
        print(f"Student {student} has overlapping courses at times: {times}")
else:
    print("No conflicts found.")


No conflicts found.
